In [1]:
import os
import time
import csv
import random

# Classe Objet
class Objet:
    def __init__(self, profit, volume):
        self.profit = profit
        self.volume = volume

# Lire les données d'une instance
def lire_fichier(filepath):
    objets = []
    try:
        with open(filepath, 'r') as file:
            N = int(file.readline().strip())  # Nombre d'objets
            capacite = int(file.readline().strip())  # Capacité du sac
            for _ in range(N):
                profit, volume = map(int, file.readline().strip().split())
                objets.append(Objet(profit, volume))
    except Exception as e:
        raise ValueError(f"Erreur dans le fichier {filepath}: {e}")
    return objets, capacite

# Calculer le profit total d'une solution
def calculer_profit(solution):
    return sum(objet.profit for objet in solution)

# Calculer le volume total d'une solution
def calculer_volume(solution):
    return sum(objet.volume for objet in solution)

# Heuristique constructive classique : Tri par ratio profit/volume
def heuristique_constructive(objets, capacite):
    objets.sort(key=lambda obj: obj.profit / obj.volume, reverse=True)
    solution = []
    capacite_actuelle = 0
    for objet in objets:
        if capacite_actuelle + objet.volume <= capacite:
            solution.append(objet)
            capacite_actuelle += objet.volume
    return solution

# Heuristique constructive randomisée
def heuristique_constructive_randomisee(objets, capacite):
    random.shuffle(objets)  # Mélange aléatoire
    return heuristique_constructive(objets, capacite)

# Recherche locale avec tabou
def recherche_locale_avec_tabou(solution_initiale, objets, capacite, tabou_size=10):
    solution = solution_initiale[:]
    tabou_list = []
    meilleur_profit = calculer_profit(solution)

    for _ in range(100):  # Nombre max d'itérations
        amelioration = False
        for objet in objets:
            if objet not in solution and objet not in tabou_list:
                voisin = solution + [objet]
                if calculer_volume(voisin) <= capacite:  # Respecter la contrainte de capacité
                    profit_voisin = calculer_profit(voisin)
                    if profit_voisin > meilleur_profit:
                        solution = voisin
                        meilleur_profit = profit_voisin
                        tabou_list.append(objet)
                        if len(tabou_list) > tabou_size:
                            tabou_list.pop(0)
                        amelioration = True
        if not amelioration:
            break
    return solution

# Méthode GRASP
def grasp(objets, capacite, max_iterations):
    meilleure_solution = []
    meilleur_profit = 0
    for _ in range(max_iterations):
        S0 = heuristique_constructive_randomisee(objets, capacite)
        S = recherche_locale_avec_tabou(S0, objets, capacite)
        profit_S = calculer_profit(S)
        if profit_S > meilleur_profit:
            meilleure_solution = S
            meilleur_profit = profit_S
    return meilleure_solution

# Lecture des instances et résolution
with open("Résultat.csv", "w", newline='') as f_write:
    writer = csv.writer(f_write, delimiter=";")
    writer.writerow(["Instance", "Profit", "Capacité utilisée", "Capacité maximale", "Temps écoulé (s)"])

    with open("all.txt", "r") as liste:
        nb_inst = int(liste.readline().strip())
        print(f"Nombre d'instances à résoudre : {nb_inst}")

        for _ in range(nb_inst):
            inst = liste.readline().strip()
            print(f"Traitement de l'instance : {inst}")

            # Vérifier si le fichier existe
            if not os.path.exists(inst):
                print(f"Fichier {inst} introuvable, instance ignorée.")
                continue

            try:
                # Lecture des données
                objets, capacite = lire_fichier(inst)
            except Exception as e:
                print(f"Erreur lors de la lecture de l'instance {inst}: {e}")
                continue

            # Résolution de l'instance avec GRASP
            start_time = time.time()
            solution = grasp(objets, capacite, 100)
            elapsed_time = time.time() - start_time

            # Calcul des résultats
            profit_solution = calculer_profit(solution)
            volume_solution = calculer_volume(solution)

            # Vérification des contraintes
            if volume_solution > capacite:
                print(f"Attention : La solution dépasse la capacité maximale ({volume_solution} > {capacite})")

            # Enregistrement dans le fichier CSV
            writer.writerow([inst, profit_solution, volume_solution, capacite, round(elapsed_time, 3)])
            print(f"Instance : {inst}, Profit : {profit_solution}, Capacité utilisée : {volume_solution}, "
                  f"Capacité maximale : {capacite}, Temps : {elapsed_time:.3f}s")
        
        print("=========================================================")


Nombre d'instances à résoudre : 25
Traitement de l'instance : test100-SC(1).txt
Instance : test100-SC(1).txt, Profit : 199, Capacité utilisée : 89, Capacité maximale : 101, Temps : 0.016s
Traitement de l'instance : test100-SC(2).txt
Instance : test100-SC(2).txt, Profit : 233, Capacité utilisée : 93, Capacité maximale : 101, Temps : 0.015s
Traitement de l'instance : test100-SC(3).txt
Instance : test100-SC(3).txt, Profit : 215, Capacité utilisée : 95, Capacité maximale : 101, Temps : 0.013s
Traitement de l'instance : test100-SC(4).txt
Instance : test100-SC(4).txt, Profit : 259, Capacité utilisée : 89, Capacité maximale : 101, Temps : 0.015s
Traitement de l'instance : test100-SC(5).txt
Instance : test100-SC(5).txt, Profit : 205, Capacité utilisée : 95, Capacité maximale : 101, Temps : 0.013s
Traitement de l'instance : test250-SC(1).txt
Instance : test250-SC(1).txt, Profit : 265, Capacité utilisée : 95, Capacité maximale : 101, Temps : 0.038s
Traitement de l'instance : test250-SC(2).txt
In